# CNN desde cero
### Organización de los datos

La creación de una red neuronal desde cero está dividida en tres notebooks:

- Organización de los datos
- Balanceo de los datos
- Creación del modelo

Este primero tiene como objetivo la creación de varios directorios que separarán las imágenes en training, validation y testing. Todas ellas deberán haber sido antes reescaladas a un tamaño reducido para evitar un coste computacional mayor durante el entramiento de la red neuronal convolucional (CNN).

Para empezar se importan algunas librerías necesarias no solo para este notebook, sino para los otros dos también.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import Model, model_from_json
from tensorflow.keras import backend

import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import os
import shutil

Se llama al directorio que contiene las imágenes originales y se crea uno nuevo con sus dos dubdirectorios para copiarlas. El objetivo de esto es realizar el escalado en este nuevo directorio para no modificar el directorio original por si surge algún problema y se requieren de las imágenes originales.

In [2]:
original_dataset_dir ='RBC_recortados'

base_dir = 'RBC_recort_reesc'
os.mkdir(base_dir)

In [3]:
esq_dir_resc = os.path.join(base_dir, 'ESQUISTOS')
os.mkdir(esq_dir_resc)
noesq_dir_resc = os.path.join(base_dir, 'NORM')
os.mkdir(noesq_dir_resc)

In [4]:
original_dataset_dir_esq = 'RBC_recortados/ESQUISTOS'
pict_dir_esq = len(os.listdir(original_dataset_dir_esq))
print('Hay {} imágenes originales de esquistocitos'.format(pict_dir_esq))

original_dataset_dir_noesq = 'RBC_recortados/NORM'
pict_dir_noesq = len(os.listdir(original_dataset_dir_noesq))
print('Hay {} imágenes originales de no esquistocitos'.format(pict_dir_noesq))

Hay 1737 imágenes originales de esquistocitos
Hay 10119 imágenes originales de no esquistocitos


Se han contado las imágenes originales disponibles: 1737 de esquistocitos y 10119 de no esquistocitos. Existe un desbalanceo de clases y, para equilibrarlas, se realizará en el siguiente notebook un balanceo híbrido (augmento de esquistocitos, disminución de no esquistocitos). Por tanto, las 1737 se copian, en cambio, del segundo grupo únicamente se usarán 3900.

In [5]:
# Copying esquistocitos pictures to directory
esq_no_copy = []

for i in range (pict_dir_esq):
    if i < 1737:
        source = os.path.join(original_dataset_dir_esq, (os.listdir(original_dataset_dir_esq))[i])
        destination = os.path.join(esq_dir_resc, (os.listdir(original_dataset_dir_esq))[i])
        shutil.copyfile(source, destination)
    else:
        esq_no_copy.append(i)
        
print ('{} imágenes de esquistocitos no se usarán'.format(len(esq_no_copy)))

0 imágenes de esquistocitos no se usarán


In [6]:
# Copying no esquistocitos pictures in directory
noesq_no_copy = []

for i in range (pict_dir_noesq):
    if i < 3900:
        source = os.path.join(original_dataset_dir_noesq, (os.listdir(original_dataset_dir_noesq))[i])
        destination = os.path.join(noesq_dir_resc, (os.listdir(original_dataset_dir_noesq))[i])
        shutil.copyfile(source, destination)
    else:
        noesq_no_copy.append(i)
        
print ('{} imágenes de no esquistocitos no se usarán'.format(len(noesq_no_copy)))

6219 imágenes de no esquistocitos no se usarán


Después de copiar las imágenes a usar, se reescalan.

In [7]:
from PIL import Image
from skimage.transform import rescale, resize

# Resized of esquistocitos pictures
esq_dir_resc = 'RBC_recort_reesc/ESQUISTOS'

for i in os.listdir(esq_dir_resc):
    path_img = esq_dir_resc+"/"+i
    img = Image.open(path_img)
    img = img.resize((220,220))
    img.save(path_img)

In [8]:
# Resized of no esquistocitos pictures
noesq_dir_resc = 'RBC_recort_reesc/NORM'

for i in os.listdir(noesq_dir_resc):
    path_img = noesq_dir_resc+"/"+i
    img = Image.open(path_img)
    img = img.resize((220,220))
    img.save(path_img)

A continuación se crea un nuevo directorio "dataset_resized" donde se incluirán tres subdirectorios: "train", "validation" y "test". En cada uno de ellos dos más: "1_esquistocitos" con las imágenes de esquistocitos y "0_no_esquistocitos" con los demás eritrocitos.

In [9]:
base_dir = 'RBC_recort_reesc'
dataset_resized = 'dataset_resized'
os.mkdir(dataset_resized)

In [16]:
# Directories for the training, validation, and test splits
train_dir = os.path.join(dataset_resized, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(dataset_resized, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(dataset_resized, 'test')
os.mkdir(test_dir)

In [17]:
# Directories with esquistocitos/no_esquistocitos pictures

# Directory with training esquistocitos/no_esquistocitos pictures
train_esq_dir = os.path.join(train_dir, '1_esquistocitos')
os.mkdir(train_esq_dir)
train_noesq_dir = os.path.join(train_dir, '0_no_esquistocitos')
os.mkdir(train_noesq_dir)

# Directory with validation esquistocitos/no_esquistocitos pictures
validation_esq_dir = os.path.join(validation_dir, '1_esquistocitos')
os.mkdir(validation_esq_dir)
validation_noesq_dir = os.path.join(validation_dir, '0_no_esquistocitos')
os.mkdir(validation_noesq_dir)

# Directory with testing esquistocitos/no_esquistocitos pictures
test_esq_dir = os.path.join(test_dir, '1_esquistocitos')
os.mkdir(test_esq_dir)
test_noesq_dir = os.path.join(test_dir, '0_no_esquistocitos')
os.mkdir(test_noesq_dir)

In [18]:
original_dataset_res_esq = 'RBC_recort_reesc/ESQUISTOS'
pict_res_esq = len(os.listdir(original_dataset_res_esq))
print('Hay {} imágenes originales de esquistocitos'.format(pict_res_esq))

original_dataset_res_noesq = 'RBC_recort_reesc/NORM'
pict_res_noesq = len(os.listdir(original_dataset_res_noesq))
print('Hay {} imágenes originales de no esquistocitos'.format(pict_res_noesq))

Hay 1737 imágenes originales de esquistocitos
Hay 3900 imágenes originales de no esquistocitos


Se copian las imágenes en las carpetas correspondientes y se cuentan cuántas hay en cada una.

In [19]:
# Copying esquistocitos pictures to directories

for i in range (pict_res_esq):
    if i < 837:
        source = os.path.join(original_dataset_res_esq, (os.listdir(original_dataset_res_esq))[i])
        destination = os.path.join(train_esq_dir, (os.listdir(original_dataset_res_esq))[i])
        shutil.copyfile(source, destination)
    elif 837 <= i < 1037:
        source = os.path.join(original_dataset_res_esq, (os.listdir(original_dataset_res_esq))[i])
        destination = os.path.join(validation_esq_dir, (os.listdir(original_dataset_res_esq))[i])
        shutil.copyfile(source, destination)
    else:
        source = os.path.join(original_dataset_res_esq, (os.listdir(original_dataset_res_esq))[i])
        destination = os.path.join(test_esq_dir, (os.listdir(original_dataset_res_esq))[i])
        shutil.copyfile(source, destination)
        

In [20]:
# Copying no esquistocitos pictures in directories

for i in range (pict_res_noesq):
    if i < 3000:
        source = os.path.join(original_dataset_res_noesq, (os.listdir(original_dataset_res_noesq))[i])
        destination = os.path.join(train_noesq_dir, (os.listdir(original_dataset_res_noesq))[i])
        shutil.copyfile(source, destination)
    elif 3000 <= i < 3200:
        source = os.path.join(original_dataset_res_noesq, (os.listdir(original_dataset_res_noesq))[i])
        destination = os.path.join(validation_noesq_dir, (os.listdir(original_dataset_res_noesq))[i])
        shutil.copyfile(source, destination)
    else:
        source = os.path.join(original_dataset_res_noesq, (os.listdir(original_dataset_res_noesq))[i])
        destination = os.path.join(test_noesq_dir, (os.listdir(original_dataset_res_noesq))[i])
        shutil.copyfile(source, destination)
        

In [21]:
# Contar cuántas imágenes hay en cada directorio
print ('Imágenes actuales de esquistocitos para training:', len(os.listdir(train_esq_dir)))
print ('Imágenes actuales de no esquistocitos para training:', len(os.listdir(train_noesq_dir)))

print ('Imágenes actuales de esquistocitos para validation:', len(os.listdir(validation_esq_dir)))
print ('Imágenes actuales de no esquistocitos para validation:', len(os.listdir(validation_noesq_dir)))

print ('Imágenes actuales de esquistocitos para testing:', len(os.listdir(test_esq_dir)))
print ('Imágenes actuales de no esquistocitos para testing:', len(os.listdir(test_noesq_dir)))


Imágenes actuales de esquistocitos para training: 837
Imágenes actuales de no esquistocitos para training: 3000
Imágenes actuales de esquistocitos para validation: 200
Imágenes actuales de no esquistocitos para validation: 200
Imágenes actuales de esquistocitos para testing: 700
Imágenes actuales de no esquistocitos para testing: 700


Como se observa hay 200 para cada clase de validación y 700 para cada clase de test. Sin embargo para el entrenamiento hay 837 imágenes de esquistocitos y 3000 de no esquistocitos.

En el siguiente notebook se aumentarán las 837 a 3000 mediance un balanceo de datos.